In [1]:
# from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime
%load_ext line_profiler
pd.options.mode.chained_assignment = None
from datetime import timedelta
import sys
sys.path.insert(0, 'E:/Credence/Projects/Ideal6-python-worker/returns/Src')
import dbUnit as db

### historical + dailyPrice export 

In [2]:
# client = MongoClient('localhost')
# db = client.Returns_WealthNFunds

# rawPrices=pd.read_csv('../excel/output/rawPrices.csv',index_col='Date',parse_dates=True)
# dividend=pd.read_csv('../excel/output/dividend.csv',index_col='Date',parse_dates=True)
# splitRatio=pd.read_csv('../excel/output/splitRatio.csv',index_col='Date',parse_dates=True)

# newRaw = pd.read_csv('excel/input/incremental/newRaw.csv',index_col='Date',parse_dates=True)
# newDividend = pd.read_csv('excel/input/incremental/newDividend.csv',index_col='Date',parse_dates=True)
# newSplit = pd.read_csv('excel/input/incremental/newSplit.csv',index_col='Date',parse_dates=True)

# rawPrices = rawPrices.append(newRaw,sort=False)
# dividend = dividend.append(newDividend,sort=False)
# splitRatio = splitRatio.append(newSplit,sort=False)

# rawPrices.to_csv('excel/output/rawPrices.csv')
# dividend.to_csv('excel/output/dividend.csv')
# splitRatio.to_csv('excel/output/splitRatio.csv')

### Returns

In [3]:
# price = pd.read_csv('../Src/excel/output/adjustedPrices.csv')

# price['Date'] = pd.to_datetime(price['Date'])

# navData = pd.read_csv('../Src/excel/input/NAV DATA.csv',parse_dates=True)

# def calcReturns(price,colDate,colSec,colPrice):
    
#     price[colDate] = pd.to_datetime(price[colDate])
#     dateDf = pd.DataFrame(index = pd.date_range(price[colDate].min(),price[colDate].max()))
    
#     distPrice = price.drop_duplicates(subset=[colSec,colDate]).reset_index().drop(columns='index')
#     pivotDistPrice = distPrice.pivot_table(index=colDate,columns=colSec,values=colPrice)
    
#     allPrice = pd.merge(dateDf,pivotDistPrice,left_index=True,right_index=True,how="left").fillna(method='ffill').fillna(method='bfill')

#     returns = allPrice.pct_change()
#     monthlyReturns = allPrice.asfreq('M').pct_change()
#     return returns,monthlyReturns

# secReturns,mR = calcReturns(price,'Date','ISIN','adjPrice')

# navReturns = calcReturns(navData,'NAVDATE','PORTCODE','NAV')

# navReturns

In [4]:
# navData['NAVDATE'] = pd.to_datetime(navData['NAVDATE'],format='%d-%m-%Y')

# distPrice = navData.drop_duplicates(subset=['PORTCODE','NAVDATE']).reset_index().drop(columns='index')

# dateDf = pd.DataFrame(data = {"Date": pd.date_range(navData['NAVDATE'].min(),navData['NAVDATE'].max())})
# dateDf.set_index('Date').loc[:'2017-05-03']

# dateDf.shape

# allPrice = pd.merge(dateDf,distPrice,left_on="Date",right_on="NAVDATE",how="left")

# distPrice['NAVDATE'].unique().shape

# allPrice.to_csv('excel/testcases/allPricesForReturns.csv')

# pivotDistPrice = distPrice.pivot_table(index='NAVDATE',columns='PORTCODE',values='NAV')
# allPrice = pd.merge(dateDf,pivotDistPrice,left_on="Date",right_on="NAVDATE",how="left")

# allPrice

## Daily Returns, Closing and group Returns 

In [2]:
def sumXactions(row):
    if row.name[1] == "Portfolio":
        row['netSumXaction'] = row.drop(["Transaction CashIn/Out"]).sum()
        row['grSumXaction'] = row.drop(["Transaction CashIn/Out","Transaction Charges","Man Fees","Administrator Fee"]).sum()
    else:
        row['grSumXaction'] = row.drop(["Cash In/Out"]).sum()
        row['netSumXaction'] = row.drop(["Cash In/Out","Transaction Charges","Man Fees","Administrator Fee"]).sum()
    return row

## Xformation Data
# for retBalSheet
def groupAsset(balSheet):
    # GROUPING balance sheet as per portfolio and assetclass
    # for portfolio
    portBalSheet = balSheet.pivot_table(index=['DATE','GROUP'],columns=['OPERATION'],values='AMOUNT',aggfunc='sum')
    # for Assets 
    balSheet['GROUP'] += ":" + balSheet['ASSET']
    portAssetBalSheet = balSheet.pivot_table(index=['DATE','GROUP'],columns=['OPERATION'],values='AMOUNT')
    balSheet = portBalSheet.append(portAssetBalSheet)
    return balSheet
# for newDailyReturns
def groupAsset2(dataframe):
    newDataframe = dataframe.copy()
    newDataframe['PORTCODE'] = dataframe['PORTCODE'].astype(str) +":"+ dataframe['ASSET_CLASS_CODE']
    dataframe = dataframe.append(newDataframe)
    dataframe['PORTCODE'] = dataframe['PORTCODE'].astype(str)
    dataframe['PORTCODE'] = dataframe['PORTCODE'].str.strip()
    return dataframe

def groupAssetOra(retBalSheet):
    # for portfolio
    portBalSheet = retBalSheet.pivot_table(index=['ASONDATE','PORTCODE'],columns=['OPERATION'],values='AMOUNT',aggfunc='sum')
    # for assets 
    retBalSheet['PORTCODE'] = retBalSheet['PORTCODE'].astype(str)
    retBalSheet['PORTCODE'] += ":"+retBalSheet['CATEGORY']
    portAssetBalSheet = retBalSheet.pivot_table(index=['ASONDATE','PORTCODE'],columns=['OPERATION'],values='AMOUNT',aggfunc='sum')
    retBalSheet = portBalSheet.append(portAssetBalSheet)
    
    return retBalSheet.unstack().stack()

In [6]:
# opening bal = filter transaction type for OPBALPURCHASE and take sum only for day one.
# todays_mtm = previous date mtm - todays date mtm
# ammortization = 0
# man fees = 0
# Xactions charges = 0
# admin fees = 0
# expense Xaction = 0
# cashIn/out = if freshcash==1 & type_of_trans==D then cashIN
#              if freshcash==1 & type_of_trans==C then cashOut
# Xaction CashIn/Out = if Xaction_type == purchase then cashOut
#                         else if Xaction_type == sell then cashIn
# Dividend  if Xaction_type == dividend then sum
#                 else 0

### test input

In [7]:
retBalSheet = pd.read_excel('../Src/excel/input/returnsBalanceSheetNEW.xlsx')
retBalSheet['DATE']=pd.to_datetime(retBalSheet['DATE'],format="%d-%b-%y")
# retBalSheet

In [8]:
# for portfolio
portBalSheet = retBalSheet.pivot_table(index=['DATE','PORTFOLIO'],columns=['OPERATION'],values='AMOUNT',aggfunc='sum')
# for assets 
retBalSheet['PORTFOLIO'] += ":"+retBalSheet['CATEGORY']
portAssetBalSheet = retBalSheet.pivot_table(index=['DATE','PORTFOLIO'],columns=['OPERATION'],values='AMOUNT')
retBalSheet = portBalSheet.append(portAssetBalSheet)
# retBalSheet = groupAsset(retBalSheet)

In [9]:
# retBalSheet = pd.read_csv('../Src/excel/output/rawInputFromOra.csv')
# retBalSheet['ASONDATE']=pd.to_datetime(retBalSheet['ASONDATE'],format="%Y-%m-%d")

### Daily Returns and closing validation worksheets Xformation

In [10]:
dailyReturns = pd.read_excel('../Src/excel/input/Pivot Daily Returns upload final.xlsx',sheet_name='Deallisting')
dailyReturns['ASONDATE']=pd.to_datetime(dailyReturns['ASONDATE'],format="%y-%m-%d")
db.insertData("dailyreturnsinput",dailyReturns)

AttributeError: module 'dbUnit' has no attribute 'insertData'

In [ ]:
writer = pd.ExcelWriter('../Src/excel/output/workingSheets.xlsx', engine = 'xlsxwriter')

#### Opening balance

In [ ]:
# newDailyReturns = dailyReturns.copy()
# newDailyReturns['PORTCODE'] =dailyReturns['PORTCODE'].astype(str) +":"+dailyReturns['ASSET_CLASS_CODE']
# dailyReturns=dailyReturns.append(newDailyReturns)
dailyReturns = groupAsset2(dailyReturns)

In [ ]:
openingBal = dailyReturns[dailyReturns['TRANSACTION_TYPE']=='OP BAL Purchase'].pivot_table(index='ASONDATE',columns='PORTCODE',values='AMOUNT',aggfunc='sum')

#### cashInOut

In [ ]:
cashFlow = pd.read_excel('../Src/excel/input/Pivot Daily Returns upload final.xlsx',sheet_name='General Banking')
cashFlow = groupAsset2(cashFlow)
openingCash = cashFlow.pivot_table(index='ASONDATE',columns='PORTCODE',values='TRANS_AMOUNT',aggfunc='sum')['500017:CASH'].loc['2017-11-30']

cashFlow = cashFlow[(cashFlow['FRESHCASH']==1)|(cashFlow['ASONDATE']=='2017-11-30')].pivot_table(index='ASONDATE',columns='PORTCODE',values='TRANS_AMOUNT',aggfunc='sum')
cashFlow.columns=pd.MultiIndex(levels=[['Cash In/Out'],['500017', '500017:CASH']],labels=[[0,0],[0,1]],names=['OPERATION','PORTFOLIO'])

cashFlow.to_excel(writer,sheet_name='Cash In-Out')
db.insertData("cashinout",cashFlow.stack().reset_index())

#### Xaction cash In/Out

In [ ]:
def XactionCashflow(row):
#     print(row['AMOUNT'])
    if (row['TRANSACTION_TYPE']!='OP BAL Purchase') & ('Purchase' in row['TRANSACTION_TYPE']):
        row['Transaction CashIn/Out'] = -row['AMOUNT']
    elif (row['TRANSACTION_TYPE']!='OP BAL Purchase') & ('Sale' in row['TRANSACTION_TYPE']):
        row['Transaction CashIn/Out'] = (row['AMOUNT'])
    elif (row['TRANSACTION_TYPE']!='OP BAL Purchase') & ('Dividend' in row['TRANSACTION_TYPE']):
        row['Dividend & Other Income'] = row['AMOUNT']
    else:
        row['Transaction CashIn/Out'] = 0
    return row

dailyReturns['Transaction CashIn/Out'] = 0
dailyReturns['Dividend & Other Income'] = 0
dailyReturns['Transaction Charges']=0
dailyReturns['Ammortization / Accreation'] = 0
dailyReturns['Man Fees'] = 0
dailyReturns['Administrator Fee'] = 0
dailyReturns['Expense Transaction'] = 0

dailyReturns=(dailyReturns.set_index('ASONDATE').apply(lambda x: XactionCashflow(x),axis=1))

dailyReturns = dailyReturns[['PORTCODE','Administrator Fee','Transaction Charges','Ammortization / Accreation','Dividend & Other Income','Expense Transaction','Man Fees','Transaction CashIn/Out']]
# dailyReturns.columns =pd.MultiIndex(levels=[['MTM'],['500017', '500017:EQ', '500017:FIS', '500017:MF']],labels=[[0,0,0,0],[0,1,2,3]])

dailyReturns.to_excel(writer,sheet_name='TransCashInOut Dividend')
db.insertData("transcashinoutdividend",dailyReturns.reset_index())

dailyReturns = dailyReturns.pivot_table(index='ASONDATE',columns='PORTCODE').fillna(0)

#### MTM

In [ ]:
positions  = pd.read_excel('../Src/excel/input/Pivot Daily Returns upload final.xlsx',sheet_name='Position')
positions = groupAsset2(positions)
positions['PORTCODE'] = positions['PORTCODE'].str.strip()

mtmdf = positions.pivot_table(index='ASONDATE',columns='PORTCODE',values='PCY_NOTIONAL_PL',aggfunc='sum')
mtmdf.to_excel(writer,sheet_name='MTM Sum')
db.insertData("mtmsum",mtmdf.stack().reset_index().rename(columns={0:"MTMSUM"}))

mtmdf.columns = pd.MultiIndex(levels=[['MTM'],['500017', '500017:EQ', '500017:FIS', '500017:MF']],labels=[[0,0,0,0],[0,1,2,3]],names=['OPERATION','PORTFOLIO'])
mtm30 = mtmdf.iloc[0]

mtmdf = mtmdf - mtmdf.shift(1)
mtmdf.iloc[0]=mtm30

mtmdf.to_excel(writer,sheet_name='MTM Diff')
db.insertData("mtmdiff",mtmdf.stack().reset_index().rename(columns={0:"MTMDIFF"}))

#### Accruals

In [ ]:
accruals  = pd.read_excel('../Src/excel/input/Pivot Daily Returns upload final.xlsx',sheet_name='Accural')
accruals = groupAsset2(accruals)
accruals = accruals.pivot_table(index='ASONDATE',columns='PORTCODE',values='ACCRUAL_AMOUNT',aggfunc='sum')
accruals.columns = pd.MultiIndex(levels=[['ACCRUAL'],['500017', '500017:EQ', '500017:FIS', '500017:MF']],labels=[[0,0,0,0],[0,1,2,3]],names=['OPERATION','PORTFOLIO'])

accruals.to_excel(writer,sheet_name='ACCRUAL Sum')
db.insertData("accrualsum",accruals.stack().reset_index())

accruals30 = accruals.iloc[0]
accruals = accruals - accruals.shift(1)
accruals.iloc[0]=accruals30

accruals.to_excel(writer,sheet_name='ACCRUAL Diff')
db.insertData("accrualdiff",accruals.stack().reset_index())

#### Merging 


In [ ]:
finalDf = mtmdf.merge(cashFlow,left_index=True,right_index=True,how='left').merge(dailyReturns,left_index=True,right_index=True,how='left').merge(accruals,left_index=True,right_index=True,how='left')
finalDf = finalDf.stack().fillna(0)

retBalSheet = finalDf
retBalSheet.to_excel(writer,sheet_name='finalMergedInputDf')
db.insertData("finalmergedinputdf",retBalSheet.reset_index())

### Oracle

,INDEX_CODE,INDEX_DATE,TRINDEX_VALUE
0,CEMENT,2017-02-03,1.00000
1,HEALTH CARE,2017-02-03,15.00000
2,Nifty 100,2017-02-03,0.10123
3,852,2017-02-03,0.00000


In [14]:
h1 = db.getDataFromMongo("rawdatareturns_nonnavport")
h1['ASONDATE']="2017-12-31"
h1['ASONDATE'] = pd.to_datetime(h1['ASONDATE'],format="%Y-%m-%d")

In [16]:
h3 = h1.copy()
for i in range(0,365*10,2):
    h2 = h1.copy()
    h2['ASONDATE'] = h2['ASONDATE']-timedelta(days=i)
    h3 = h3.append(h2);

In [18]:
db.insertDataToMongo("rawdatareturns_nonnavport",h3)

In [3]:
# Get Historical data from Mongo
histData = db.getDataFromMongo("rawdatareturns_nonnavport")
if (histData.empty!=True):
    histData['ASONDATE']=pd.to_datetime(histData['ASONDATE'],format="%d-%b-%y")
    dailyRaw = db.getDataFromOracle("dailyRawData")
    dailyRaw['ASONDATE']=pd.to_datetime(dailyRaw['ASONDATE'],format="%d-%b-%y")
    retBalSheet = histData.append(dailyRaw,sort=False)
    db.insertDataToMongo("rawdatareturns_nonnavport",retBalSheet)
else:
    print("dataNotFound")
    db.insertDataToMongo("rawdatareturns_nonnavport",db.getDataFromOracle("dailyRawData"))

In [4]:
retBalSheet = groupAssetOra(retBalSheet)

In [5]:
rettest = retBalSheet.reset_index().set_index(['ASONDATE'])

forPort =rettest[~rettest['PORTCODE'].str.contains(':')].reset_index().set_index(["ASONDATE","PORTCODE"])
forNonPort = rettest[rettest['PORTCODE'].str.contains(':')].reset_index().set_index(["ASONDATE","PORTCODE"])

# netSUM:
netPortSum = forPort.drop(['Transaction CashIn/Out'],1).sum(1)
netNonPortSum = forNonPort.drop(["Cash In/Out","Transaction Charges","Man Fees","Administrator Fee"],1).sum(1)
netSum = pd.concat([netPortSum,netNonPortSum],axis=1).fillna(0).sum(1)
# GrossSUM:
grPortSum = forPort.drop(["Transaction CashIn/Out","Transaction Charges","Man Fees","Administrator Fee"],1).sum(1)
grNonPortSum = forNonPort.drop(["Cash In/Out"],1).sum(1)
grSum = pd.concat([grPortSum,grNonPortSum],axis=1).fillna(0).sum(1)


rettest = rettest.reset_index().set_index(["ASONDATE","PORTCODE"])
rettest['netSumXaction'] = netSum
rettest['grSumXaction'] = grSum

In [6]:
rettest

OPERATION                Accrual  Administrator Fee  \
ASONDATE   PORTCODE                                   
2008-01-05 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2008-01-07 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2008-01-09 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2008-01-11 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2008-01-13 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
...                          ...                ...   
2017-12-25 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2017-12-27 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2017-12-29 50001            56.0                6.0   
           50001:Bond       28.0                3.0   
           50001:Equity     28.0                3.0   
           50027            56.0                6.0   
           50027:Bond       28.0                3.0   
           50027:Equity     28.0                3.0   
2017-12-31 50001           112.0               12.0   
           50001:Bond       56.0                6.0   
           50001:Equity     56.0                6.0   
           50027           112.0               12.0   
           50027:Bond       56.0                6.0   
           50027:Equity     56.0                6.0   
2018-01-01 50001           448.0               48.0   
           50001:Bond      224.0               24.0   
           50001:Equity    224.0               24.0   
           50027           448.0               48.0   
           50027:Bond      224.0               24.0   
           50027:Equity    224.0               24.0   

OPERATION                Ammortization / Accreation  Cash In/Out  \
ASONDATE   PORTCODE                                                
2008-01-05 50001                               20.0       2000.0   
           50001:Bond                          10.0       1000.0   
           50001:Equity                        10.0       1000.0   
           50027                               20.0       2000.0   
           50027:Bond                          10.0       1000.0   
           50027:Equity                        10.0       

### Calculations

#### Note:
- **Daily Returns**
    - get previous days closing.
    - Calculate daily Returns and closing balance for a specified date
    - save Daily Returns to mongo
- **Group returns**
    - get historical data from mongo
    - calc group returns
    - save groupReturns to mongo

#### Closing 

In [7]:
# balSheet = retBalSheet.apply(lambda row: sumXactions(row),axis=1)

In [8]:
balSheet=rettest

In [9]:
closingBal = pd.DataFrame(index=balSheet.index)
closingBal = closingBal.assign(**{"GROPEN":np.zeros(balSheet.index.size),"NETOPEN":np.zeros(balSheet.index.size),"GRCLOSE":np.zeros(balSheet.index.size),"NETCLOSE":np.zeros(balSheet.index.size),"GRRETURNS":np.zeros(balSheet.index.size),"NETRETURNS":np.zeros(balSheet.index.size),"LOGGRRETURNS":np.zeros(balSheet.index.size),"LOGNETRETURNS":np.zeros(balSheet.index.size)})

closingBal = closingBal.unstack('PORTCODE')

# openingBalValues = np.insert(openingBal.values[0],1,openingCash)
closingBal['GROPEN'].iloc[0]=0
closingBal['NETOPEN'].iloc[0]=0

# closingBal['GROPEN'].iloc[0]=openingBalValues
# closingBal['NETOPEN'].iloc[0]=openingBalValues

balSheetUnstacked = balSheet.unstack()

closingBal['NETCLOSE'].values[:] = closingBal['NETOPEN'].values[0] + balSheetUnstacked['netSumXaction'].cumsum().values
closingBal['GRCLOSE'].values[:] = closingBal['GROPEN'].values[0] + balSheetUnstacked['grSumXaction'].cumsum().values
closingBal['NETOPEN'].values[1:] = closingBal['NETCLOSE'].values[:-1]
closingBal['GROPEN'].values[1:] = closingBal['GRCLOSE'].values[:-1]

closingBal['NETRETURNS'] = closingBal['NETCLOSE'].pct_change()
closingBal['GRRETURNS'] = closingBal['GRCLOSE'].pct_change()

closingBal['LOGGRRETURNS']=np.log(closingBal['GRCLOSE'].pct_change()+1)
closingBal['LOGNETRETURNS']=np.log(closingBal['NETCLOSE'].pct_change()+1)

# closingBal['LOGGRRETURNS'] = np.log(closingBal['GRCLOSE'])-np.log(closingBal['GRCLOSE'].shift(1))
# closingBal['LOGNETRETURNS'] = np.log(closingBal['NETCLOSE'])-np.log(closingBal['NETCLOSE'].shift(1))
# closingBal

In [10]:
allDateClosing = pd.DataFrame(index=pd.date_range(closingBal.index.min(),closingBal.index.max()))
closingBal = pd.concat([allDateClosing,closingBal],axis=1)
# closingBal = allDateClosing.merge(closingBal,left_index=True,right_index=True,how='left')
closingBal.columns = pd.MultiIndex.from_tuples(closingBal.columns)
closingBal = closingBal.fillna(method='ffill')
# closingBal['netClose']

In [11]:
closingBal

GROPEN                                                \
                50001 50001:Bond 50001:Equity      50027 50027:Bond   
2008-01-05        0.0        0.0          0.0        0.0        0.0   
2008-01-06        0.0        0.0          0.0        0.0        0.0   
2008-01-07     2680.0     1351.0       1351.0     2480.0     1251.0   
2008-01-08     2680.0     1351.0       1351.0     2480.0     1251.0   
2008-01-09     5360.0     2702.0       2702.0     4960.0     2502.0   
2008-01-10     5360.0     2702.0       2702.0     4960.0     2502.0   
2008-01-11     8040.0     4053.0       4053.0     7440.0     3753.0   
2008-01-12     8040.0     4053.0       4053.0     7440.0     3753.0   
2008-01-13    10720.0     5404.0       5404.0     9920.0     5004.0   
2008-01-14    10720.0     5404.0       5404.0     9920.0     5004.0   
2008-01-15    13400.0     6755.0       6755.0    12400.0     6255.0   
2008-01-16    13400.0     6755.0       6755.0    12400.0     6255.0   
2008-01-17    16080.0     8106.0       8106.0    14880.0     7506.0   
2008-01-18    16080.0     8106.0       8106.0    14880.0     7506.0   
2008-01-19    18760.0     9457.0       9457.0    17360.0     8757.0   
2008-01-20    18760.0     9457.0       9457.0    17360.0     8757.0   
2008-01-21    21440.0    10808.0      10808.0    19840.0    10008.0   
2008-01-22    21440.0    10808.0      10808.0    19840.0    10008.0   
2008-01-23    24120.0    12159.0      12159.0    22320.0    11259.0   
2008-01-24    24120.0    12159.0      12159.0    22320.0    11259.0   
2008-01-25    26800.0    13510.0      13510.0    24800.0    12510.0   
2008-01-26    26800.0    13510.0      13510.0    24800.0    12510.0   
2008-01-27    29480.0    14861.0      14861.0    27280.0    13761.0   
2008-01-28    29480.0    14861.0      14861.0    27280.0    13761.0   
2008-01-29    32160.0    16212.0      16212.0    29760.0    15012.0   
2008-01-30    32160.0    16212.0      16212.0    29760.0    15012.0   
2008-01-31    34840.0    17563.0      17563.0    32240.0    16263.0   
2008-02-01    34840.0    17563.0      17563.0    32240.0    16263.0   
2008-02-02    37520.0    18914.0      18914.0    34720.0    17514.0   
2008-02-03    37520.0    18914.0      18914.0    34720.0    17514.0   
...               ...        ...          ...        ...        ...   
2017-12-03  4850800.0  2445310.0    2445310.0  4488800.0  2264310.0   
2017-12-04  4850800.0  2445310.0    2445310.0  4488800.0  2264310.0   
2017-12-05  4853480.0  2446661.0    2446661.0  4491280.0  2265561.0   
2017-12-06  4853480.0  2446661.0    2446661.0  4491280.0  2265561.0   
2017-12-07  4856160.0  2448012.0    2448012.0  4493760.0  2266812.0   
2017-12-08  4856160.0  2448012.0    2448012.0  4493760.0  2266812.0   
2017-12-09  4858840.0  2449363.0    2449363.0  4496240.0  2268063.0   
2017-12-10  4858840.0  2449363.0    2449363.0  4496240.0  2268063.0   
2017-12-11  4861520.0  2450714.0    2450714.0  4498720.0  2269314.0   
2017-12-12  4861520.0  2450714.0    2450714.0  4498720.0  2269314.0   
2017-12-13  4864200.0  2452065.0    2452065.0  4501200.0  2270565.0   
2017-12-14  4864200.0  2452065.0    2452065.0  4501200.0  2270565.0   
2017-12-15  4866880.0  2453416.0    2453416.0  4503680.0  2271816.0   
2017-12-16  4866880.0  2453416.0    2453416.0  4503680.0  2271816.0   
2017-12-17  4869560.0  2454767.0    2454767.0  4506160.0  2273067.0   
2017-12-18  4869560.0  2454767.0    2454767.0  4506160.0  2273067.0   
2017-12-19  4872240.0  2456118.0    2456118.0  4508640.0  2274318.0   
2017-12-20  4872240.0  2456118.0    2456118.0  4508640.0  2274318.0   
2017-12-21  4874920.0  2457469.0    2457469.0  4511120.0  2275569.0   
2017-12-22  4874920.0  2457469.0    2457469.0  4511120.0  2275569.0   
2017-12-23  4877600.0  2458820.0    2458820.0  4513600.0  2276820.0   
2017-12-24  4877600.0  2458820.0    2458820.0  4513600.0  2276820.0   
2017-12-25  4880280.0  2460171.0    2460171.0  4516080.0  2278071.0   
2017-12-26  4880280.0  2460171.0    246

In [12]:
# closingBal.stack().to_excel(writer,sheet_name='Closing')
# db.insertData("dailyclosingreturns",closingBal.stack().reset_index())

In [13]:
# writer.save()
# writer.close()

In [14]:
# def calcDateRangeReturns(date):
#     date = pd.to_datetime(date,format='%Y-%m-%d')
#     dateData={
#      '1D':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(days=1):date].sum(),
#      '1W':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=1):date].sum(),
#      '1FN':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=2):date].sum(),
#      '1M':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=4):date].sum(),
#      '3M':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=12):date].sum(),
#      '6M':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=24):date].sum(),
#      '9M':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=36):date].sum(),
#      '1Y':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=48):date].sum(),
#      '2Y':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=96):date].sum(),
#      '3Y':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=144):date].sum(),
#      '5Y':closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=240):date].sum(),
#      'INCEPTION':closingBal[['logGrReturns','logNetReturns']].loc[:date].sum()
#     }
#     dateRangeReturns = pd.DataFrame(index=pd.MultiIndex(levels=[['logGrReturns', 'logNetReturns'], ['Portfolio', 'Portfolio:Bond', 'Portfolio:Equity']],labels=[[0,0,0,1,1,1],[0,1,2,0,1,2]]),data=dateData)

# #     dateRangeReturns = pd.DataFrame(index=['Portfolio','Portfolio:Bond','Portfolio:Equity'],data=dateData)
    
#     return dateRangeReturns
    

#### Group returns

In [15]:
def calcDateRangeReturns2(date):
    date = pd.to_datetime(date,format='%Y-%m-%d')
    dateData={
     '1D':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('1D').sum().iloc[-2:].sum().values,
     '1W':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('1W').sum().iloc[-2:].sum().values,
     '1FN':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('2W').sum().iloc[-2:].sum().values,
     '1M':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('1M').sum().iloc[-2:].sum().values,
     '3M':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('3M').sum().iloc[-2:].sum().values,
     '6M':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('6M').sum().iloc[-2:].sum().values,
     '9M':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('9M').sum().iloc[-2:].sum().values,
     '1Y':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('1Y').sum().iloc[-2:].sum().values,
     '2Y':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('2Y').sum().iloc[-2:].sum().values,
     '3Y':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('3Y').sum().iloc[-2:].sum().values,
     '5Y':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].resample('5Y').sum().iloc[-2:].sum().values,
     'INCEPTION':closingBal[['LOGGRRETURNS','LOGNETRETURNS']][:date].sum().values
    }
    dateRangeReturns = pd.DataFrame(index=pd.MultiIndex.from_product([['LOGGRRETURNS', 'LOGNETRETURNS'],closingBal.columns.levels[1]]),data=dateData)
#     dateRangeReturns = pd.DataFrame(index=['Portfolio','Portfolio:Bond','Portfolio:Equity'],data=dateData)
    
    return np.exp(dateRangeReturns)-1

In [16]:
def calcDateRangeReturns3(date,col):
    date = pd.to_datetime(date,format='%Y-%m-%d')
    dateData={
     '1D':closingBal[col][:date].resample('1D').sum().iloc[-2:].sum().values,
     '1W':closingBal[col][:date].resample('1W').sum().iloc[-2:].sum().values,
     '1FN':closingBal[col][:date].resample('2W').sum().iloc[-2:].sum().values,
     '1M':closingBal[col][:date].resample('1M').sum().iloc[-2:].sum().values,
     '3M':closingBal[col][:date].resample('3M').sum().iloc[-2:].sum().values,
     '6M':closingBal[col][:date].resample('6M').sum().iloc[-2:].sum().values,
     '9M':closingBal[col][:date].resample('9M').sum().iloc[-2:].sum().values,
     '1Y':closingBal[col][:date].resample('1Y').sum().iloc[-2:].sum().values,
     '2Y':closingBal[col][:date].resample('2Y').sum().iloc[-2:].sum().values,
     '3Y':closingBal[col][:date].resample('3Y').sum().iloc[-2:].sum().values,
     '5Y':closingBal[col][:date].resample('5Y').sum().iloc[-2:].sum().values,
     'INCEPTION':closingBal[col][:date].sum().values
    }
    dateRangeReturns = pd.DataFrame(index=pd.MultiIndex.from_product([col,closingBal.columns.levels[1]]),data=dateData)
#     dateRangeReturns = pd.DataFrame(index=['Portfolio','Portfolio:Bond','Portfolio:Equity'],data=dateData)
    
    return np.exp(dateRangeReturns)-1

In [25]:
groupReturns = calcDateRangeReturns3('2018-01-01',['LOGGRRETURNS','LOGNETRETURNS'])
groupReturns['TYPE']="PORTFOLIO"

In [26]:
db.insertDataToMongo("groupreturns",groupReturns.reset_index(),"2018-01-01")
db.insertDataToOracle("groupreturns",groupReturns.reset_index(),type="PORTFOLIO")


In [ ]:
date = '2018-08-07'
date = pd.to_datetime(date,format='%Y-%m-%d')
# closingBal['logGrReturns'].loc[date-datetime.timedelta(days=7):date].sum()

closingBal[['logGrReturns','logNetReturns']].loc[date-datetime.timedelta(weeks=200):date].sum()